
<img src="https://katiehouse3.pythonanywhere.com/static/moviechat/img/movie.png" width="25px" align="left">

<h1>Finetune Hollybot GPT-2 on Microsoft Azure</h1>


This script will use Microsoft Azure's amlcompute to finetune Hollybot's [GPT-2 transformer model](https://huggingface.co/transformers/model_doc/gpt2.html).

## What is Hollybot ?
Hollybot is a chatbot trained on over 220,579 conversations from movie scripts. The end result is a chatbot that learns the 

In [35]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.62


In [36]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep='\n')

In [11]:
import os
import shutil

project_folder = './pytorch-gpt2'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('processed_data_final.csv', project_folder)
shutil.copy('dialogue_data.pkl', project_folder)

'./pytorch-gpt2/dialogue_data.pkl'

In [4]:
'''
datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                      datastore_name="gpt2training",
                                                        account_name="amherstwstorageinnganzr",
                                                    container_name="gpt2training", 
                                                      account_key='<MY ACCOUNT KEY>',
                                                      create_if_not_exists=True)

'''

'\ndatastore = Datastore.register_azure_blob_container(workspace=ws, \n                                                      datastore_name="gpt2training",\n                                                        account_name="amherstwstorageinnganzr",\n                                                    container_name="gpt2training", \n                                                      account_key=\'<MY ACCOUNT KEY>\',\n                                                      create_if_not_exists=True)\n'

In [14]:

import azureml.data
from azureml.data.azure_storage_datastore import AzureFileDatastore, AzureBlobDatastore
datastore = Datastore.get(ws, datastore_name='gpt2training')

datastore.upload(src_dir='./pytorch-gpt2',
                 target_path='pytorch-gpt2',
                 overwrite=True,
                 show_progress=True)

Uploading an estimated of 2 files
Uploading ./pytorch-gpt2/dialogue_data.pkl
Uploading ./pytorch-gpt2/processed_data_final.csv
Uploaded ./pytorch-gpt2/processed_data_final.csv, 1 files out of an estimated total of 2
Uploaded ./pytorch-gpt2/processed_data_final.csv, 1 files out of an estimated total of 2
Uploaded ./pytorch-gpt2/dialogue_data.pkl, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_0e7399f6908d4f1dbe15d7c726feba64

Uploaded ./pytorch-gpt2/dialogue_data.pkl, 2 files out of an estimated total of 2


In [15]:
print("These are your availible datastores...")
import azureml.core
from azureml.core import Workspace, Datastore
datastores = ws.datastores
for name, ds in datastores.items():
    print(name, ds.datastore_type)

These are your availible datastores...
workspaceblobstore AzureBlob
workspacefilestore AzureFile
commercial_blocks AzureBlob
machine_translation AzureBlob
commercialblockclassification AzureBlob
gpt2training AzureBlob


In [16]:
#get named datastore from current workspace
datastore = Datastore.get(ws, datastore_name='gpt2training')
ws.set_default_datastore('gpt2training')

In [17]:
datastore.as_mount()

$AZUREML_DATAREFERENCE_gpt2training

In [26]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

'''
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    compute_target.delete()
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    
'''
compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC24', min_nodes=2, 
                                                       max_nodes=4)

# create the cluster
compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Creating
Succeeded..............
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 2, 'targetNodeCount': 2, 'nodeStateCounts': {'preparingNodeCount': 2, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-12-08T23:38:09.692000+00:00', 'errors': None, 'creationTime': '2019-12-08T23:36:34.117992+00:00', 'modifiedTime': '2019-12-08T23:36:51.443726+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 2, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC24'}


In [27]:
from azureml.core import Experiment

experiment_name = 'pytorch-gpt2'
experiment = Experiment(ws, name=experiment_name)

In [28]:
import os
import shutil

project_folder = './train-on-amlcompute'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train.py', project_folder)
shutil.copy('processed_data_final.csv', project_folder)

'./train-on-amlcompute/processed_data_final.csv'

In [29]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment("myenv")

myenv.docker.enabled = True
myenv.python.conda_dependencies = CondaDependencies.create(pip_packages=['utils','torch','tensorflow','azureml-sdk','argparse','pandas','numpy','transformers', 'scikit-learn'])

In [30]:
from azureml.train.dnn import PyTorch

script_params = {
    '--data_dir': datastore.path('/pytorch-gpt2').as_mount(),
    '--output_dir': './outputs'
}


estimator = PyTorch(source_directory=project_folder, 
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='train.py',
                    use_gpu=True,
                    inputs=[datastore.path('/gpt2training').as_download(),datastore.as_mount()],
                    pip_packages=['pillow==5.4.1','git+https://github.com/huggingface/transformers'])


# Set environment
estimator.run_config.environment = myenv

WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.
WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.
WARNING - framework_version is not specified, defaulting to version 1.2.
WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.


In [31]:
run = experiment.submit(estimator)
print(run)

WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.
Submitting /Users/katiehouse/Documents/github/chatbot-with-personality-2/models/train-on-amlcompute directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


Run(Experiment: pytorch-gpt2,
Id: pytorch-gpt2_1575848361_16f023ca,
Type: azureml.scriptrun,
Status: Queued)


In [32]:
print(run.get_details())

{'runId': 'pytorch-gpt2_1575848361_16f023ca', 'target': 'gpu-cluster', 'status': 'Queued', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '1f225141-f143-4e85-876f-6e0a3534da15', 'azureml.git.repository_uri': 'https://github.com/katiehouse3/chatbot-with-personality.git', 'mlflow.source.git.repoURL': 'https://github.com/katiehouse3/chatbot-with-personality.git', 'azureml.git.commit': 'd1a23dbb7f90d7638eb419b6bb98d8a4a2b0c2a4', 'mlflow.source.git.commit': 'd1a23dbb7f90d7638eb419b6bb98d8a4a2b0c2a4', 'azureml.git.dirty': 'True', 'AzureML.DerivedImageName': 'azureml/azureml_17c5d7f9f15131cc7ae45d39982e34c8', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'runDefinition': {'script': 'train.py', 'arguments': ['--data_dir', '$AZUREML_DATAREFERENCE_a327617aecfa4f2a89468c63e534ddd3', '--output_dir', './outputs'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target'

In [34]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 'sdk_version': '1.0.62'})